In [1]:
# Data manipulation
import numpy as np
import pandas as pd

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os

# Performance metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

from preprocessing import create_and_preprocess_X_y, create_train_test_set
from model import create_y_pred_baseline, calculate_metric, baseline_metric_score


# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import LSTM, Flatten, Dense

2023-09-13 15:22:28.531560: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-13 15:22:28.534911: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 15:22:28.610386: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 15:22:28.613120: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 15:22:29.840623: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
metrics_baseline = baseline_metric_score()
metrics_baseline

0.04497379454476886

In [3]:
# Load and preprocess the data using create_and_preprocess_X_y()
X, y = create_and_preprocess_X_y()

In [4]:
X.shape, y.shape

((88, 7400, 106), (88, 100, 106))

In [5]:
# Split the data into train and test sets using create_train_test_set()
X_train, X_test, y_train, y_test = create_train_test_set(X, y, train_size=0.8)

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((70, 7400, 106), (18, 7400, 106), (70, 100, 106), (18, 100, 106))

In [7]:
# Define the LSTM model
model = Sequential([
    layers.Masking(mask_value=-1, input_shape=(X_train.shape[1], X_train.shape[2])),
    layers.LSTM(units=64, activation ='tanh', return_sequences=True),
    layers.LSTM(units=64, activation ='tanh', return_sequences=False),
    layers.RepeatVector(100),
    layers.TimeDistributed(layers.Dense(units=106, activation = "softmax"))
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 7400, 106)         0         
                                                                 
 lstm (LSTM)                 (None, 7400, 64)          43776     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 repeat_vector (RepeatVecto  (None, 100, 64)           0         
 r)                                                              
                                                                 
 time_distributed (TimeDist  (None, 100, 106)          6890      
 ributed)                                                        
                                                                 
Total params: 83690 (326.91 KB)
Trainable params: 83690 

In [9]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32) 

Epoch 1/10
3/3 [==============================] - 58s 16s/step - loss: 12.4625 - accuracy: 0.0143
Epoch 2/10
3/3 [==============================] - 47s 15s/step - loss: 12.1937 - accuracy: 0.0150
Epoch 3/10
3/3 [==============================] - 47s 16s/step - loss: 11.5021 - accuracy: 0.0150
Epoch 4/10
3/3 [==============================] - 48s 16s/step - loss: 11.1120 - accuracy: 0.0150
Epoch 5/10
3/3 [==============================] - 48s 16s/step - loss: 10.8806 - accuracy: 0.0150
Epoch 6/10
3/3 [==============================] - 50s 15s/step - loss: 10.7293 - accuracy: 0.0150
Epoch 7/10
3/3 [==============================] - 52s 17s/step - loss: 10.6354 - accuracy: 0.0150
Epoch 8/10
3/3 [==============================] - 49s 16s/step - loss: 10.5670 - accuracy: 0.0150
Epoch 9/10
3/3 [==============================] - 51s 17s/step - loss: 10.5313 - accuracy: 0.0143
Epoch 10/10
3/3 [==============================] - 49s 16s/step - loss: 10.4973 - accuracy: 0.0137


In [11]:
# Make predictions test 
y_test_pred = model.predict(X_test)

1/1 [==============================] - 6s 6s/step


In [12]:
model_score = calculate_metric(y_test_pred, y_test)

In [13]:
model_score

0.9988852634790757

In [14]:
y_test_pred

array([[[0.00046358, 0.00100522, 0.00060932, ..., 0.00049613,
         0.00119841, 0.00059873],
        [0.00046358, 0.00100522, 0.00060932, ..., 0.00049613,
         0.00119841, 0.00059873],
        [0.00046358, 0.00100522, 0.00060932, ..., 0.00049613,
         0.00119841, 0.00059873],
        ...,
        [0.00046358, 0.00100522, 0.00060932, ..., 0.00049613,
         0.00119841, 0.00059873],
        [0.00046358, 0.00100522, 0.00060932, ..., 0.00049613,
         0.00119841, 0.00059873],
        [0.00046358, 0.00100522, 0.00060932, ..., 0.00049613,
         0.00119841, 0.00059873]],

       [[0.00045798, 0.00099682, 0.00060569, ..., 0.0004923 ,
         0.00118734, 0.00059175],
        [0.00045798, 0.00099682, 0.00060569, ..., 0.0004923 ,
         0.00118734, 0.00059175],
        [0.00045798, 0.00099682, 0.00060569, ..., 0.0004923 ,
         0.00118734, 0.00059175],
        ...,
        [0.00045798, 0.00099682, 0.00060569, ..., 0.0004923 ,
         0.00118734, 0.00059175],
        [0.0